Generate setigen signals that can then be put into seticore and used as part of clustering

In [5]:
%matplotlib inline

import numpy as np
import setigen as stg
import blimpy as bl
import matplotlib.pyplot as plt
from astropy import units as u
from astropy.coordinates import Angle
import os
import random
import h5py

# Create output directories
output_file = 'et_signals/test.h5'
os.makedirs('et_signals', exist_ok=True)
os.makedirs('outputs', exist_ok=True)

# Frame configuration
fchans = 1024
tchans = 32
df = 2.7939677238464355 * u.Hz
dt = 18.253611008 * u.s
fch1 = random.uniform(6000, 6100) * u.MHz  # Random start frequency

# Generate synthetic noise
frame = stg.Frame(
    fchans=fchans * u.pixel,
    tchans=tchans * u.pixel,
    df=df,
    dt=dt,
    fch1=fch1
)
frame.add_noise(x_mean=10, noise_type='chi2')

# Frequency range for injection
f_start_min = frame.get_frequency(index=0)
f_start_max = frame.get_frequency(index=fchans - 1)
f_start = random.uniform(f_start_min, f_start_max)

# Add signal
frame.add_signal(
    path=stg.constant_path(
        f_start=f_start,
        drift_rate=random.uniform(-2, 2) * u.Hz / u.s
    ),
    t_profile=stg.constant_t_profile(
        level=frame.get_intensity(snr=random.uniform(10, 30))
    ),
    f_profile=stg.gaussian_f_profile(
        width=random.uniform(20, 60) * u.Hz
    ),
    bp_profile=stg.constant_bp_profile(level=1)
)

# Create realistic BL-style header
header = {
    'DIMENSION_LABELS': np.array([b'time', b'feed_id', b'frequency'], dtype=object),
    'az_start': 0.0,
    'data_type': 1,
    'fch1': fch1.value,  # MHz
    'foff': -df.to_value(u.MHz),  # negative MHz
    'machine_id': 20,
    'nbits': 32,
    'nchans': fchans,
    'nifs': 1,
    'source_name': 'synthetic',
    'src_dej': Angle(0 * u.deg),
    'src_raj': Angle('0h0m0s'),
    'telescope_id': 6,
    'tsamp': dt.value,  # seconds
    'tstart': 60000,
    'za_start': 0.0,
    'nfpc': fchans
}

# Convert to float32 and save using from_data
data_array = frame.data.astype('<f4')  # little-endian float32
frame = stg.Frame.from_data(
    df=df,
    dt=dt,
    fch1=fch1,
    ascending=False,
    data=data_array,
    metadata=header
)
frame.save_h5(output_file)

# Plot waterfall
plt.clf()
fb = bl.Waterfall(output_file)
fb.plot_waterfall()
plt.savefig('outputs/plot.png')


blimpy.waterfall INFO     __write_to_hdf5_light: Writing the spectra matrix for et_signals/test.h5 without blobbing.
hdf5plugin._filters WARNING  Deprecation: hdf5plugin.Bitshuffle's lz4 argument is deprecated, use cname='lz4' or 'none' instead.
hdf5plugin._filters WARNING  Deprecation: hdf5plugin.Bitshuffle's lz4 argument is deprecated, use cname='lz4' or 'none' instead.
blimpy.waterfall INFO     Conversion time: 0.01sec


In [6]:
!seticore "et_signals/test.h5" --output "seticore_output/test.dat"

welcome to seticore, version 1.0.6
running in dedoppler mode.
loading input from et_signals/test.h5
dedoppler parameters: max_drift=10.00 min_drift=0.0000 snr=25.00
writing output to seticore_output/test.dat
unable to infer coarse channel size for data with dimensions: 32 x 1024. please set the nfpc header.
